# tutorial_TensorBoard
https://www.tensorflow.org/tensorboard/get_started?hl=ja

In [4]:
import tensorflow as tf
from tensorflow import keras
import datetime

In [2]:
# load mnist 
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# model定義のための関数
def create_model():
      return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model.fit()を実行するとtf.keras.callbacks.TensorBoard コールバックによってログが作成され, 保存される.

In [3]:
# model
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # logの保存先
# histogram_freq=1にするとエポックごとにヒストグラム計算される
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3631 - accuracy: 0.8943 - val_loss: 0.0982 - val_accuracy: 0.9708
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0989 - accuracy: 0.9699 - val_loss: 0.0855 - val_accuracy: 0.9741
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0701 - accuracy: 0.9778 - val_loss: 0.0738 - val_accuracy: 0.9763
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0509 - accuracy: 0.9829 - val_loss: 0.0682 - val_accuracy: 0.9788
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0406 - accuracy: 0.9872 - val_loss: 0.0598 - val_accuracy: 0.9814


Terminalで次のコマンドを実行
```
tensorboard --logdir logs/fit
```

# TensorBoardで画像を描画する

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

In [6]:
# fashion_mnistをload
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [7]:
# 画像のshape
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


In [8]:
# batch_size, height, width, channels
img = np.reshape(train_images[0], (-1, 28, 28, 1))

In [11]:
logdir = "logs/train_data/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)
with file_writer.as_default():
    tf.summary.image("Training data", img, step=0)

ターミナルで次のコマンドを実行
```
tensorboard --logdir logs/train_data
```

# 任意の画像データのログ

In [13]:
import io

In [14]:
logdir = "logs/plots/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def image_grid():
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(10,10))
    for i in range(25):
        # Start next subplot.
        plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)

    return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Training data", plot_to_image(figure), step=0)